In [1]:
import pybullet as p
import time
import math
from datetime import datetime

pybullet build time: Jun  2 2022 15:41:26


In [1]:
%run env_tester.py

pybullet build time: Jun  2 2022 15:41:26


STARTING SPOT TEST ENV


GL error 0x500 detected in glGenBuffers


error: Cannot load URDF file.

In [2]:
p.connect(p.GUI)
# p.connect(p.DIRECT)

GL error 0x500 detected in glGenBuffers


0

In [3]:
# class Dog:
p.resetSimulation()
plane = p.loadURDF("plane.urdf")
p.setGravity(0,0,-9.8)
p.setTimeStep(1./500)
p.setRealTimeSimulation(1)
#p.setDefaultContactERP(0)
#urdfFlags = p.URDF_USE_SELF_COLLISION+p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
urdfFlags = p.URDF_USE_SELF_COLLISION
quadruped = p.loadURDF("data/a1/urdf/a1.urdf",[0,0,0.48],[0,0,0,1], flags = urdfFlags,useFixedBase=0)
quadNumJoints = p.getNumJoints(quadruped)

In [11]:
#enable collision between lower legs
# for j in range (quadNumJoints):
#         print(p.getJointInfo(quadruped,j))

#    2, b'FR_hip_joint'    - JOINT
#    3, b'FR_thigh_joint'  - JOINT
#    4, b'FR_calf_joint'   - JOINT
#    5, b'FR_foot_fixed'   - END-EFFECTOR
#    6, b'FL_hip_joint'    - JOINT
#    7, b'FL_thigh_joint'  - JOINT
#    8, b'FL_calf_joint'   - JOINT
#    9, b'FL_foot_fixed'   - END-EFFECTOR
#    10, b'RR_hip_joint'   - JOINT
#    11, b'RR_thigh_joint' - JOINT
#    12, b'RR_calf_joint'  - JOINT
#    13, b'RR_foot_fixed'  - END-EFFECTOR
#    14, b'RL_hip_joint'   - JOINT
#    15, b'RL_thigh_joint' - JOINT
#    16, b'RL_calf_joint'  - JOINT
#    17, b'RL_foot_fixed'  - END-EFFECTOR

end_effectors = [5, 9, 13, 17]

# DOESNT MAKE SENSE:
# lower_legs = [2,5,8,11]
# for l0 in lower_legs:
#     for l1 in lower_legs:
#         if (l1>l0):
#             enableCollision = 1
#             #print("collision for pair",l0,l1, p.getJointInfo(quadruped,l0)[12],p.getJointInfo(quadruped,l1)[12], "enabled=",enableCollision)
#             p.setCollisionFilterPair(quadruped, quadruped, 2,5,enableCollision)

jointIds=[]
paramIds=[]

maxForceId = p.addUserDebugParameter("maxForce",0,100,20)

for j in range (quadNumJoints):
    p.changeDynamics(quadruped,j,linearDamping=0, angularDamping=0)
    info = p.getJointInfo(quadruped,j)
#     print(info)
    jointType = info[2]
    if (jointType==p.JOINT_PRISMATIC or jointType==p.JOINT_REVOLUTE):
        jointIds.append(j)

# print(jointIds)

p.getCameraImage(480,320)


joints=[]
t = 0  #time set to 0
prev_pos = p.getBasePositionAndOrientation(quadruped)
print(prev_pos)

FR_pos = p.getJointState(quadruped, end_effectors[0])
FL_pos = p.getJointState(quadruped, end_effectors[1])
BR_pos = p.getJointState(quadruped, end_effectors[2])
BL_pos = p.getJointState(quadruped, end_effectors[3])
# print(FR_pos, FL_pos, BR_pos, BL_pos)

((0.00010253809997560786, 9.861559936208766e-05, 0.4199906222058692), (0.0002094486443328408, -3.4019549392802184e-07, -1.565393805729106e-05, 0.9999999779430517))


In [ ]:
#gait pattern w/o limits
useRealTimeSimulation = 1

#phase of each leg
phi_FR = 0
phi_FL = 4.*math.pi / 4
phi_BR = 4.*math.pi / 4
phi_BL = 0.*math.pi / 4
phi = [phi_FR, phi_FL, phi_BR, phi_BL]

leg_FR = [2, 3, 4]
leg_FL = [6, 7, 8]
leg_BR = [10, 11, 12]
leg_BL = [14, 15, 16]
joint_Id = [leg_FR, leg_FL, leg_BR, leg_BL]

# motor ranges without swing ratio
thigh_min = 3/16*math.pi
thigh_range = 4/16*math.pi - thigh_min

calf_min = -6/16*math.pi
calf_range = -5/16*math.pi - calf_min 

# motor ranges with swing ratio
thigh_swing_min = 6/32*math.pi
thigh_swing_range = 7/32*math.pi - thigh_swing_min

calf_swing_min = -6/16*math.pi
calf_swing_range = -11/32*math.pi - calf_swing_min
thigh_stance_min = 7/32*math.pi
thigh_stance_range = 8/32*math.pi - thigh_stance_min

calf_stance_min = -11/32*math.pi
calf_stance_range = -10/32*math.pi - calf_stance_min


while(1):
    
    #sets the time frequency of simulation
    if (useRealTimeSimulation):
        magnitude = 100000
        period = 10 # a multiple of 100
        dt = datetime.now()
        time = (dt.microsecond + (dt.second * 1000000))
        t = (((time / magnitude) % period) / period) * 2. * math.pi # gets an angle between 0 and 2pi 
    else: 
        if(t >= 2.*math.pi):
            t = 0
        else:
            t = t + (2.*math.pi)/12000
        p.stepSimulation()        
    
    # each leg can be treated as a 2-dof robotic arm: governing equations:
    #     x = l1 * cos(theta1) + l2 * cos(theta1 + theta2)
    #     y = l1 * sin(theta1) + l2 * sin(theta1 + theta2)
    # theoretical inverse kinematics: 
    #     theta2 = math.acos((pow(x, 2) + pow(y, 2) - pow(l1, 2) - pow(l2, 2))/(2*l1*l2)
    #     theta1 = math.atan(y/x) - math.atan((l2*math.sin(theta2))/(l1 + l2*math.cos(theta2)))
    
    # calf motor range: -1/4pi to -7/8pi (usable range: -1/4pi to -7/8pi)
    # thigh motor range: -1/4pi to 11/8pi (usable range: -1/4pi to 1/2pi)
    
    #calculates gait cycle for each leg separately according to each phase
    for i in range(4):
    
    # translating position in degrees to motor positions:            

        proportion1 = thigh_range / 2 # divide by the y range of sin
        theta1 = proportion1 * (math.sin(t + phi[i]) + 1) + thigh_min # offset the sin curve so it is all positive

        proportion2 = calf_range / 2
        theta2 = proportion2 * (math.cos(t + phi[i]) + 1) + calf_min

        motor_angles = [0., theta1, theta2] 
        motor_velocity_slow = [0., .01, .01]
        motor_velocity_fast = [0., .02, .02]
        
        motor_force = [0, .005, .005]

        p.setJointMotorControlArray(quadruped,
                       joint_Id[i], 
                       p.POSITION_CONTROL, 
                       motor_angles, 
                       motor_force)

    # need to separate the range of motion into "stance" and "swing"

#         swing_ratio = .75
#         stance_ratio = 1.-swing_ratio
        
#         motor_force = [0, .005, .005]

#         if((math.sin(t + phi[i])) < (swing_ratio*2*math.pi)): #slow down when touching the ground, speed up when off the ground
       
#             proportion_stance = 2*math.pi / (2*math.pi*stance_ratio)
#             t_stance = proportion * (t + phi[i]) + (2*math.pi - 2*math.pi*stance_ratio)

#             proportion_thigh_stance = thigh_stance_range /2
#             theta_thigh_stance = proportion_thigh_stance * (math.sin(t_stance)) + thigh_stance_min

#             proportion_calf_stance = calf_stance_range /2
#             theta_calf_stance = proportion_calf_stance * (math.cos(t_stance))  + calf_stance_min

#             motor_angles_stance = [0., theta_thigh_stance, theta_calf_stance] 

#             p.setJointMotorControlArray(quadruped,
#                                    joint_Id[i], 
#                                    p.POSITION_CONTROL, 
#                                    motor_angles_stance, 
#                                    motor_force)
#         else: 
#             proportion_swing = 2*math.pi / (2*math.pi*swing_ratio)
#             t_swing = proportion * (t + phi[i]) + (2*math.pi - 2*math.pi*swing_ratio)
            
#             proportion_thigh_swing = thigh_swing_range /2
#             theta_thigh_swing = proportion_thigh_swing * (math.sin(t_swing)) + thigh_swing_min

#             proportion_calf_swing = calf_swing_range /2
#             theta_calf_swing = proportion_calf_swing * (math.cos(t_swing)) + calf_swing_min

#             motor_angles_swing = [0., theta_thigh_swing, theta_calf_swing]

#             p.setJointMotorControlArray(quadruped,
#                                    joint_Id[i], 
#                                    p.POSITION_CONTROL, 
#                                    motor_angles_swing, 
#                                    motor_force)
        


#         if(i == 1):
#             print(theta1)
        
    

In [7]:
p.resetSimulation()

In [4]:
p.disconnect()